# Swedish sign alphabet recognition 
Lukas Lönnroth

2020




Mount the drive to witch you have uploaded the corpus to.

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
from zipfile import ZipFile
file_name = 'drive/My Drive/train_test.zip'

with ZipFile(file_name, 'r')as zip:
  zip.extractall()
  print('Done')

In [0]:
%tensorflow_version 1.x

In [0]:
import os
import tensorflow as tf
import os
import glob
import numpy as np
import time
from cv2 import cv2
import matplotlib.pyplot as plt
import pickle

import random
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow import keras

Name the model so that logs are easy to understand

In [0]:
IMG_SIZE = 64
MODE = 'canny'
DATA_DIR = 'train_test'
NAME = '{}_{}-{}_128drop05_256_512_dense_64_{}'.format(MODE, IMG_SIZE, IMG_SIZE, int(time.time()))

In [0]:
LOG_DIR='drive/My Drive/logs/{}'.format(NAME)
tensorboard = TensorBoard(log_dir=LOG_DIR)

In [0]:
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [0]:
train_x = pickle.load( open( "{}/train_x.p".format(DATA_DIR), "rb" ) )
train_y = pickle.load( open( "{}/train_y.p".format(DATA_DIR), "rb" ) )
test_x = pickle.load( open( "{}/test_x.p".format(DATA_DIR), "rb" ) )
test_y = pickle.load( open( "{}/test_y.p".format(DATA_DIR), "rb" ) )

In [0]:
print(str(len(train_x))+ ' ' + str(len(test_x)))
print(str(len(train_y))+ ' ' + str(len(test_y)))

In [0]:
plt.imshow(test_x[1], cmap='gray')
print(alphabet[test_y[1]])

In [0]:
def resize_reshape(arr, size, mode):
  new_array = []
  for img in arr:
    img = cv2.resize(img, (size, size))
    if mode == 'canny':
      img = cv2.Canny(img, 70, 90)
    new_array.append(img)
  plt.imshow(new_array[1], cmap='gray')
  return np.array(new_array).reshape(-1, size, size, 1)

In [0]:
train_x = resize_reshape(train_x, IMG_SIZE, MODE)
test_x = resize_reshape(test_x, IMG_SIZE, MODE)
train_y = np.array(train_y)
test_y = np.array(test_y)

In [0]:
train_x = train_x/255.0
test_x = test_x/255.0

In [0]:
train_x.shape

### Defining the model


In [0]:
model = Sequential()

model.add(Conv2D(128, (5, 5), input_shape=train_x.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(26))
model.add(Activation('sigmoid'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
model.summary()

### Training the model

In [0]:
history = model.fit(train_x, train_y, batch_size=5, epochs=10, validation_split=0.1, callbacks=[tensorboard])

In [0]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
acc = history.history['acc']
val_acc = history.history['val_acc']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [0]:
model.evaluate(test_x, test_y)

Save the model.

In [0]:
model_name = NAME + '.model'
model.save(model_name)
print(NAME + '.model' + ' saved..')

In [0]:
model.evaluate(test_x, test_y)